<a href="https://colab.research.google.com/github/SingTown/openmv_tensorflow_training_scripts/blob/main/openmv_mobilenet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train MobilenetV2 and Save to OpenMV

This Code is for TensorFlow 2

## Download Cats vs Dogs dataset from github


In [1]:
!git clone --branch master --depth=1 https://github.com/haritha91/Cats-Dogs-Classifier---Keras.git cat_dog

Cloning into 'cat_dog'...
remote: Enumerating objects: 10018, done.
remote: Counting objects: 100% (10018/10018), done.
remote: Compressing objects: 100% (10015/10015), done.
remote: Total 10018 (delta 3), reused 10017 (delta 3), pack-reused 0
Receiving objects: 100% (10018/10018), 224.07 MiB | 26.24 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (10011/10011), done.


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras, lite

CLASS_NUM = 2 # dog and cat
INPUT_SIZE = 96 #image size is 96 * 96 *3

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, #All the pixel values would be 0-1
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('cat_dog/dataset/training_set',
                          target_size = (INPUT_SIZE, INPUT_SIZE),
                          batch_size = 32)

test_set = test_datagen.flow_from_directory('cat_dog/dataset/test_set',
                        target_size = (INPUT_SIZE, INPUT_SIZE),
                        batch_size = 32)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Define Model

alpha=0.35 for lower size

Note: Do NOT use GlobalAveragePooling2D Which is NOT supported by OpenMV

In [4]:
base = keras.applications.MobileNetV2(
    include_top=False,
    alpha=0.35,
    weights="imagenet",
    input_shape=(INPUT_SIZE, INPUT_SIZE, 3)
)
base.trainable = False

x = base.output
flatten = keras.layers.Flatten()(x) 
predictions = keras.layers.Dense(CLASS_NUM, activation='softmax')(flatten)
model = keras.models.Model(inputs=base.input, outputs=predictions)
#print(model.summary())

2019640/2019640 [==============================] - 0s 0us/step


## Train model
After 1 epoches, accuracy is about 88% 

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(training_set, epochs = 1, validation_data = test_set)


250/250 [==============================] - 71s 271ms/step - loss: 0.4979 - accuracy: 0.8382 - val_loss: 0.4409 - val_accuracy: 0.8745


## Full Integer Quantization

ref: https://www.tensorflow.org/lite/performance/post_training_quantization#full_integer_quantization

In [6]:
quant_set = test_datagen.flow_from_directory('cat_dog/dataset/test_set',
                        target_size = (INPUT_SIZE, INPUT_SIZE),
                        batch_size = 1)
def representative_dataset():
    for i in range(100):
        x, y = quant_set.next()
        yield [x]

# Convert the tflite.
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()

# Save the model.
with open('trained.tflite', 'wb') as f:
  f.write(tflite_quant_model)

Found 2000 images belonging to 2 classes.


/usr/local/lib/python3.8/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


## Succeed

Copy trained.tflite to OpenMV4 H7 Plus, run this code in OpenMV4 H7 Plus: https://github.com/SingTown/openmv_tensorflow_training_scripts/blob/main/main.py